## Modelo de lenguaje con tokenización por caracteres

### Consigna

  Seleccionar un corpus de texto sobre el cual entrenar el modelo de lenguaje.

  Realizar el pre-procesamiento adecuado para tokenizar el corpus, estructurar el dataset y separar entre datos de entrenamiento y validación.

  Proponer arquitecturas de redes neuronales basadas en unidades recurrentes para implementar un modelo de lenguaje.

  Con el o los modelos que consideren adecuados, generar nuevas secuencias a partir de secuencias de contexto con las estrategias de greedy search y beam search determístico y estocástico. En este último caso observar el efecto de la temperatura en la generación de secuencias.

### Sugerencias

  Durante el entrenamiento, guiarse por el descenso de la perplejidad en los datos de validación para finalizar el entrenamiento. Para ello se provee un callback.

  Explorar utilizar SimpleRNN (celda de Elman), LSTM y GRU.
  rmsprop es el optimizador recomendado para la buena convergencia. No obstante se pueden explorar otros.

In [ ]:
import random
import io
import pickle

import os
import platform
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from scipy.special import softmax
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# descargar de textos.info
import urllib.request

# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs

In [ ]:
import torch

# Verificar disponibilidad de CUDA
if torch.cuda.is_available():
    print("✅ ¡ÉXITO! PyTorch detectó tu GPU.")
    print(f"Modelo de GPU: {torch.cuda.get_device_name(0)}")
    print(f"Versión de CUDA: {torch.version.cuda}")
else:
    print("⚠️ PyTorch NO detectó la GPU. Se usará el procesador (CPU).")

✅ ¡ÉXITO! PyTorch detectó tu GPU.
Modelo de GPU: NVIDIA GeForce RTX 4070 Laptop GPU
Versión de CUDA: 12.8


In [ ]:
raw_html = urllib.request.urlopen('https://www.textos.info/daniel-defoe/robinson-crusoe/ebook')
raw_html = raw_html.read()

# Parsear artículo, 'lxml' es el parser a utilizar
article_html = bs.BeautifulSoup(raw_html, 'lxml')

# Encontrar todos los párrafos del HTML (bajo el tag <p>)
# y tenerlos disponible como lista
article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text + ' '

# pasar todo el texto a minúscula
article_text = article_text.lower()

In [ ]:
# en article text se encuentra el texto de todo el libro
article_text[:1000]

' nací en 1632,\nen la ciudad de york, de una buena familia, aunque no de la región,\npues mi padre era un extranjero de brema que, inicialmente, se\nasentó en hull. allí consiguió hacerse con una considerable fortuna\ncomo comerciante y, más tarde, abandonó sus negocios y se fue a\nvivir a york, donde se casó con mi madre, que pertenecía a la\nfamilia robinson, una de las buenas familias del condado de la cual\nobtuve mi nombre, robinson kreutznaer. mas, por la habitual\nalteración de las palabras que se hace en inglaterra, ahora nos\nllaman y nosotros también nos llamamos y escribimos nuestro nombre\ncrusoe; y así me han llamado siempre mis compañeros. tenía dos hermanos mayores, uno de ellos fue coronel de un\nregimiento de infantería inglesa en flandes, que antes había estado\nbajo el mando del célebre coronel lockhart, y murió en la batalla\nde dunkerque contra los españoles. lo que fue de mi segundo\nhermano, nunca lo he sabido al igual que mi padre y mi madre\ntampoco supieron l